# Script to Create Explanations to Recommendations - [GOOGLE COLAB]

In [1]:
# Install missing libraries
!pip install boto3

In [2]:
#@title Import Libraries
import os
import pickle
import numpy as np
import pandas as pd
import random
import itertools
import json
import time
import boto3

In [3]:
# Bedrock test
AWS_ACCESS_KEY_ID=''
AWS_SECRET_ACCESS_KEY=''

## Define LLM Task to create Explanations

In [5]:
# Bedrock Test
class ExplainBot:
    def __init__(self, model_name="mistral.mistral-7b-instruct-v0:2"):
        """
        Initialize the ChatBot with AWS Bedrock
        """
        self.model_name = model_name

        # Initialize AWS Bedrock client
        self.bedrock_runtime = boto3.client(
            service_name="bedrock-runtime",
            region_name="us-east-1",
            aws_access_key_id=AWS_ACCESS_KEY_ID,
            aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        )

    def get_response(self, prompt):
        """
        Generate a response from AWS Bedrock based on the given prompt
        """
        try:
            payload = {
                "modelId": self.model_name,
                "contentType": "application/json",
                "accept": "application/json",
                "body": json.dumps({
                    "prompt": prompt,
                    "max_tokens": 300,
                    "temperature": 0.3,
                    "top_p": 0.9
                }),
            }

            # Invoke model
            response = self.bedrock_runtime.invoke_model(**payload)

            # Read response
            response_body = json.loads(response["body"].read())

            # Extract text
            completion = response_body.get("outputs", [{}])[0].get("text", "").strip()

            # print(f"Prompt: {prompt}") # USE FOR TESTING ONLY

            return prompt, completion if completion else "Error: Empty response from Bedrock."

        except Exception as e:
            print(f"Error in get_response: {e}")
            return "Error: Unable to generate a response."

    def chat_recommendation_explanation(self, input_data, books_df):
        """
        Format the prompt and send it to AWS Bedrock for explanation generation
        """
        try:
            # Extract user data
            instance = input_data['user_data']['instances'][0]

            user_id = instance['user_id']
            liked_books = instance.get('liked_books', {})
            disliked_books = instance.get('disliked_books', {})
            liked_genres = list(instance.get('liked_genres', {}).keys())
            disliked_genres = instance.get('disliked_genres', [])
            liked_authors = instance.get('liked_authors', [])
            disliked_authors = instance.get('disliked_authors', [])
            additional_preferences = instance.get('additional_preferences', '')

            # Retrieve the recommendation details
            recommendation, cosine_similarity = input_data['recommendation'][0]

            # Retrieve book details from books_df
            book_info = books_df[books_df["title"] == recommendation]
            if not book_info.empty:
                book_author = book_info["author"].values[0]
                book_publish_year = int(book_info["publish_year"].values[0])
                book_genres = book_info["genre_consolidated"].values[0]
                book_description = book_info["description"].values[0]
                top_reviews = book_info["reviews"].values[0]
            else:
                book_author = "Unknown Author"
                book_publish_year = "Unknown Year"
                book_genres = "Unknown Genre"
                book_description = "No description available"
                top_reviews = "No reviews available"

            # Retrieve info for liked books
            liked_book_titles = list(liked_books.keys())
            liked_books_info = books_df[books_df["title"].isin(liked_book_titles)]
            liked_books_descriptions = {
                row["title"]: row["description"] for _, row in liked_books_info.iterrows()
            }

            # Retrieve info for disliked books
            disliked_book_titles = list(disliked_books.keys())
            disliked_books_info = books_df[books_df["title"].isin(disliked_book_titles)]
            disliked_books_descriptions = {
                row["title"]: row["description"] for _, row in disliked_books_info.iterrows()
            }

            # Format liked books with descriptions
            liked_books_str = "\n".join(
                f"- {title} (Rated {liked_books[title].get('rating', 'N/A')}): {liked_books_descriptions.get(title, 'No description available')}"
                for title in liked_books
            ) if liked_books else "None"

            # Format disliked books with descriptions
            disliked_books_str = "\n".join(
                f"- {title} (Rated {disliked_books[title].get('rating', 'N/A')}): {disliked_books_descriptions.get(title, 'No description available')}"
                for title in disliked_books
            ) if disliked_books else "None"

            # System instructions
            content_begin = (
                "\n\nHuman:"
                "You are a book reading analyst. Your job is to generate a direct and concise explanation for why a specific book was recommended to the user.\n"

                "Please provide a recommendation explanation that focuses on the key aspects of the user’s preferences that align with the recommended book. The explanation should avoid repeating all the genres the user likes if they do not align with the book. Instead, focus on the similarities between the user’s actual interests and the recommended book, as well as any relevant themes, genres, or topics that match the user's profile. Only mention genres, themes, or categories directly related to the book. If the recommendation is based on collaborative filtering or a low similarity score, clarify that the suggestion comes from a broader range of similar users rather than direct content match.\n"

                "### Important Considerations:\n"
                "  - Be mindful of the **context and themes** of the recommended book.\n"
                "  - If the book covers **sensitive topics** (e.g., war, trauma, historical events), ensure that your explanation is **empathetic, respectful, and avoids trivialization**.\n"
                "  - If the book deals with **serious or tragic themes**, acknowledge the gravity of the subject in your response.\n"
                "  - Do **not** include generic statements like 'This is a great book!'—instead, focus on why it aligns with the user’s reading preferences.\n"
                "\n\n"

                "Do **not** include:\n"
                "    - A summary of your response.\n"
                "    - Meta information (e.g., 'This explanation is under 300 tokens').\n"
                "    - Any mention of task constraints.\n"

                "Additionally, you may incorporate insights from the **top reader review** of the recommended book to support the reasoning. The top review is selected based on a combination of review score and helpfulness, and includes:\n"
                "  - **review_text** (reader's written thoughts),\n"
                "  - **review_score** (numerical rating),\n"
                "  - **review_helpfulness** (an indicator of how useful other readers found the review).\n"

                "**Provide ONLY the explanation itself**.\n"

                "The response **MUST BE UNDER 300 TOKENS**.\n\n"

                "\n"

                "These are examples of the outputs we expect:\n"
                "\n"
                "### Example 1\n"
                "**User Data:**\n"
                "- **Favorite Genres:** Fiction, Classics\n"
                "- **Disliked Genres:** Fantasy\n"
                "- **Favorite Authors:** F. Scott Fitzgerald, Harper Lee\n"
                "- **Disliked Authors:** J.R.R. Tolkien\n"
                "- **Liked Books:**\n"
                "  - The Great Gatsby (Rated 5): A novel about wealth, love, and the disillusionment of the American Dream.\n"
                "  - To Kill a Mockingbird (Rated 4): A powerful story about race, morality, and justice in the Deep South.\n"
                "- **Disliked Books:**\n"
                "  - The Lord of the Rings (Rated 2): A high-fantasy epic with extensive world-building and mythical storytelling.\n"
                "- **Additional Preferences:**\n"
                "  - I don't like to read books that are way too verbose. I like smooth, engaging reads.\n"
                "\n"
                "**Recommended Book:** The Age of Innocence\n"
                  "- **Cosine Similarity:** 0.87\n"
                  "- **Author:** J.D. Salinger\n"
                  "- **Publish Year:** 1951\n"
                  "- **Genre:** Classic, Fiction, Coming-of-Age\n"
                  "- **Description:** A novel that follows Holden Caulfield, a disillusioned teenager who struggles with identity, societal expectations, and the phoniness of the adult world.\n"
                  "- **Top Review:** “Holden's frustration with the adult world felt painfully real. The writing is simple but piercing—every word carries weight, and the emotional honesty hit me hard.” (Score: 5, Helpfulness: 91)\n"
                "\n"
                "- **Recommendation Explanation:** *The Age of Innocence* was recommended because it reflects your appreciation for emotionally resonant classics that critique societal norms, much like *The Great Gatsby* and *To Kill a Mockingbird*. Its concise, impactful prose fits your preference for smooth, engaging reads. The top review highlights the novel’s emotional honesty and clarity—qualities that complement your dislike of verbosity and your draw toward meaningful storytelling.\n"
                "\n---\n"
                "### Example 2\n"
                "**User Data:**\n"
                "- **Favorite Genres:** Mystery, Thriller\n"
                "- **Disliked Genres:** Romance\n"
                "- **Favorite Authors:** Agatha Christie, Arthur Conan Doyle\n"
                "- **Liked Books:**\n"
                "  - The Hound of the Baskervilles (Rated 5): A Sherlock Holmes mystery involving a supernatural legend and deductive reasoning.\n"
                "  - Murder on the Orient Express (Rated 5): A classic Agatha Christie whodunit featuring Hercule Poirot.\n"
                "- **Disliked Books:**\n"
                "  - The Notebook (Rated 1): A sentimental love story centered on rekindled romance.\n"
                "- **Additional Preferences:**\n"
                "  - I enjoy books with intricate plots, clever twists, and strong detective work.\n"
                "\n"
                "**Recommended Book:** Gone Girl\n"
                  "- **Cosine Similarity:** 0.81\n"
                  "- **Author:** Gillian Flynn\n"
                  "- **Publish Year:** 2012\n"
                  "- **Genre:** Psychological Thriller, Mystery\n"
                  "- **Description:** A suspenseful novel that follows the mysterious disappearance of Amy Dunne, unraveling secrets, deception, and unreliable narration.\n"
                  "- **Top Review:** “Absolutely gripping. The twists kept coming, and just when I thought I had it figured out, everything flipped. A masterclass in unreliable narration and psychological tension.” (Score: 5, Helpfulness: 14)\n"
                "\n"
                "- **Recommendation Explanation:** *Gone Girl* was recommended because it’s a psychological thriller built on clever misdirection and layered tension—hallmarks of the mysteries you love. Like *Murder on the Orient Express*, it keeps you guessing with intricately constructed twists. The top review highlights its expert use of unreliable narration and gripping suspense, aligning perfectly with your preference for stories full of surprise and intellectual challenge.\n"
                "\n---\n"
                "### Example 3 (Collaborative Filtering)\n"
                "**User Data:**\n"
                "- **Favorite Genres:** Science Fiction, Dystopian\n"
                "- **Disliked Genres:** Historical Fiction\n"
                "- **Favorite Authors:** Isaac Asimov, Philip K. Dick\n"
                "- **Liked Books:**\n"
                "  - 1984 (Rated 5): A dystopian novel exploring totalitarianism, surveillance, and free will.\n"
                "  - Do Androids Dream of Electric Sheep? (Rated 4): A sci-fi classic questioning the nature of humanity and artificial intelligence.\n"
                "- **Disliked Books:**\n"
                "  - The Pillars of the Earth (Rated 2): A historical epic centered on medieval cathedral-building and political intrigue.\n"
                "- **Additional Preferences:**\n"
                "  - I prefer thought-provoking narratives that explore deep philosophical and societal themes.\n"
                "\n"
                "**Recommended Book:** The Road\n"
                  "- **Cosine Similarity:** 0.26\n"
                  "- **Author:** Cormac McCarthy\n"
                  "- **Publish Year:** 2006\n"
                  "- **Genre:** Post-Apocalyptic, Dystopian Fiction\n"
                  "- **Description:** A bleak yet moving story of a father and son struggling for survival in a post-apocalyptic world, grappling with morality and despair.\n"
                  "- **Top Review:** “I found this book relentlessly grim and emotionally draining. The sparse writing style didn’t connect with me, and the plot felt too bleak to be meaningful.” (Score: 2, Helpfulness: 89)\n"
                "\n"
                "- **Recommendation Explanation:** *The Road* was recommended because it shares your interest in dystopian fiction that probes moral and existential questions, much like *1984* and *Do Androids Dream of Electric Sheep?*. While the top review critiques its bleak tone and minimalist style, those very elements may resonate with your appreciation for thought-provoking, philosophical storytelling in post-apocalyptic settings.\n"
                "\n---\n"
                "### Example 4 (Fantasy Fans, Content Similarity)\n"
                "**User Data:**\n"
                "- **Favorite Genres:** Fantasy, Adventure\n"
                "- **Disliked Genres:** Horror\n"
                "- **Favorite Authors:** J.K. Rowling, Brandon Sanderson\n"
                "- **Liked Books:**\n"
                "  - Harry Potter series (Rated 5): A coming-of-age fantasy about magic, friendship, and heroism.\n"
                "  - Mistborn (Rated 5): A high-fantasy novel with a unique magic system and political intrigue.\n"
                "- **Disliked Books:**\n"
                "  - It (Rated 1): A horror novel featuring supernatural terror and psychological fear elements.\n"
                "- **Additional Preferences:**\n"
                "  - I prefer thought-provoking narratives that explore deep philosophical and societal themes.\n"
                "\n"
                "**Recommended Book:** The Priory of the Orange Tree\n"
                  "- **Cosine Similarity:** 0.56\n"
                  "- **Author:** Samantha Shannon\n"
                  "- **Publish Year:** 2019\n"
                  "- **Genre:** Fantasy, Adventure, High Fantasy\n"
                  "- **Description:** A sweeping epic featuring powerful queens, dragons, and political intrigue, blending elements of adventure and deep world-building.\n"
                  "- **Top Review:** 'A bit slow at times, but ultimately rewarding. The world-building is intricate and the characters are complex. It’s more about the political intricacies than the action, which might not appeal to everyone.' (Score: 4, Helpfulness: 91)\n"
                "\n"
                "- **Recommendation Explanation:** *The Priory of the Orange Tree* was recommended based on the principle of collaborative filtering. While it doesn’t directly match your specific preferences for fast-paced, action-heavy fantasy, readers who enjoyed *Mistborn* and *Harry Potter* often appreciated its deep world-building and intricate political plotlines. This recommendation aims to offer a book with rich narrative complexity that might expand your reading horizons and challenge your typical reading style, similar to how other users with similar tastes have enjoyed it despite some differences in genre focus.\n"
                "\n---\n"
                "### Example 5 (Self-Improvement, Content Similarity)\n"
                "**User Data:**\n"
                "- **Favorite Genres:** Non-Fiction, Self-Improvement\n"
                "- **Disliked Genres:** Fantasy, Sci-Fi\n"
                "- **Favorite Authors:** James Clear, Malcolm Gladwell\n"
                "- **Liked Books:**\n"
                "  - Atomic Habits (Rated 5): A practical guide to building good habits and breaking bad ones.\n"
                "  - Outliers (Rated 4): A study on success and the factors that contribute to high achievement.\n"
                "- **Disliked Books:**\n"
                "  - The Hobbit (Rated 1): A high-fantasy adventure about a reluctant hero and a dragon quest.\n"
                "- **Additional Preferences:**\n"
                "  - I prefer books with practical advice and real-world applications rather than abstract theories.\n"
                "\n"
                "**Recommended Book:** The Power of Habit\n"
                  "- **Cosine Similarity:** 0.98\n"
                  "- **Author:** Charles Duhigg\n"
                  "- **Publish Year:** 2012\n"
                  "- **Genre:** Non-Fiction, Self-Improvement, Psychology\n"
                  "- **Description:** A book exploring the science behind habit formation, providing practical strategies for making lasting behavioral changes.\n"
                  "- **Top Review:** 'A transformative read that breaks down the science of habits in an accessible way. While it's a bit repetitive at times, the practical steps it offers are life-changing.' (Score: 5, Helpfulness: 92)\n"
                "\n"
                "- **Recommendation Explanation:** *The Power of Habit* was recommended because it closely aligns with your interest in self-improvement books like *Atomic Habits*. This book shares similar themes of habit-building and offers practical, real-world advice that matches your preferences for actionable content. It also explores the psychology behind habits, providing additional insights that can complement your existing reading. While the books share a similar focus, this recommendation also considers other readers' preferences, which further strengthens its relevance to your reading goals.\n"
                "\n---\n"
                "Now, based on the user data provided below, generate a similar explanation.\n"
            )

            # User Preferences
            content_user_profile = "### User Profile Information:\n"
            content_user_profile += f"- **Favorite Genres:** {', '.join(liked_genres) if liked_genres else 'None'}\n"
            content_user_profile += f"- **Disliked Genres:** {', '.join(disliked_genres) if disliked_genres else 'None'}\n"
            content_user_profile += f"- **Favorite Authors:** {', '.join(liked_authors) if liked_authors else 'None'}\n"
            content_user_profile += f"- **Disliked Authors:** {', '.join(disliked_authors) if disliked_authors else 'None'}\n"
            content_user_profile += f"- **Liked Books:**\n{liked_books_str}\n"
            content_user_profile += f"- **Disliked Books:**\n{disliked_books_str}\n"
            content_user_profile += f"- **User's Additional Preferences:** {additional_preferences}\n"

            # Recommended Book
            content_recommendation = f"\n### Recommended Book: {recommendation}\n"
            content_recommendation += f"- **Cosine Similarity:** {cosine_similarity}\n"
            content_recommendation += f"- **Genre:** {book_genres}\n"
            content_recommendation += f"- **Author:** {book_author}\n"
            content_recommendation += f"- **Publish Year:** {book_publish_year}\n"
            content_recommendation += f"- **Description:** {book_description}\n"
            content_recommendation += f"- **Top Review:** {top_reviews}\n"

            # Task Instructions
            content_request = (
                "\n### Task: Provide a Direct Explanation\n"
                "Now explain concisely why this book was recommended based on the user's data provided.\n"
                "Mention both liked and disliked aspects if relevant. Also consider the user's self-provided additional preferences and compare it to the recommended book data.\n"
                "Keep the response focused, avoiding unnecessary words.\n"
                "Do NOT include greetings, introductions, or phrases like 'Here’s why...' or 'Sure, here is...'.\n"
                "Start directly with the explanation, speaking directly to the user.\n\n"
                "The response **MUST BE UNDER 300 TOKENS**.\n\n"
                "Assistant:"
            )

            # Combine all content sections
            content = content_begin + content_user_profile + content_recommendation + content_request

            # Generate explanation
            prompt, explanation_text = self.get_response(content)
            explanation_text = explanation_text.strip()

            # Create JSON response
            response_json = {
                "recommended_book": recommendation,
                "author": book_author,
                "description": book_description,
                "explanation": explanation_text,
                "prompt": prompt
            }

            return response_json

        except Exception as e:
            print(f"Error in chat_recommendation_explanation: {e}")
            return "Error: Unable to generate a response."

## Create Test Data

In [6]:
input_data = {
  "user_data": {
    "instances": [
      {
        "user_id": "1025",
        "liked_books": {
          "Hatchet": {
            "title": "Hatchet",
            "author": "Gary Paulsen",
            "genre": "Young Adult / Survival",
            "rating": 5
          },
          "On Combat: The Psychology and Physiology of Deadly Conflict in War and Peace": {
            "title": "On Combat: The Psychology and Physiology of Deadly Conflict in War and Peace",
            "author": "Dave Grossman",
            "genre": "Psychology / Military",
            "rating": 4
          },
          "Life, the Universe and Everything": {
            "title": "Life, the Universe and Everything",
            "author": "Douglas Adams",
            "genre": "Fiction / Dystopian",
            "rating": 5
          }
        },
        "disliked_books": {
          "Twilight": {
            "title": "Twilight",
            "author": "Stephenie Meyer",
            "genre": "Young Adult / Paranormal Romance",
            "rating": 1
          },
          "Magic & Madness in the Library : Protagonists Among the Stacks": {
            "title": "Magic & Madness in the Library : Protagonists Among the Stacks",
            "author": "Rhonda Byrne",
            "genre": "Self-Help / Spirituality",
            "rating": 1
          }
        },
        "liked_genres": {
          "Military History": "keep",
          "Survival": "keep",
          "Romantic Comedy": "keep"
        },
        "disliked_genres": [
          "Paranormal Romance",
          "Spirituality"
        ],
        "liked_authors": [
          "Gary Paulsen",
          "Graeme Simsion"
        ],
        "disliked_authors": [
          "Stephenie Meyer",
          "Rhonda Byrne"
        ],
        "additional_preferences": "I’m drawn to realism, self-reliance themes, and characters with unusual or methodical personalities. I don’t mind humor, but it has to feel grounded.",
        "authors": 0,
        "categories": 0,
        "description": 0,
        "target_book": 0,
        "target_book_rating": 0
      }
    ]
  },
  "recommendation": [
    ["Tempting Fate", 0.73]
  ]
}

## Connect to DB for Book Data

In [7]:
import psycopg2 as ps

In [8]:
host_name = "booksdataclean.cu9jy7bp7ol8.us-east-1.rds.amazonaws.com"
dbname = 'booksfull'
port = '5432'
username = 'postgres'
password = ''

In [9]:
def connect_to_db(host_name, dbname, port, username, password):
  try:
    conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

  except ps.OperationalError as e:
    raise e

  else:
    print("Connected!")
    return conn

conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [10]:
query = """
    WITH ranked_reviews AS (
        SELECT
            title,
            review_text,
            review_score,
            review_helpfulness,
            ROW_NUMBER() OVER (PARTITION BY title ORDER BY review_helpfulness DESC) AS RankNum
        FROM books_info
    ),

    top_reviews AS (
        SELECT
            title,
            CONCAT(review_text, ' (Score: ', review_score, ', Helpful: ', review_helpfulness, ')') AS formatted_review
        FROM ranked_reviews
        WHERE RankNum = 1
    )

    SELECT
        b.title,
        b.author,
        b.publish_year,
        b.genre_consolidated,
        b.description,
        STRING_AGG(t.formatted_review, ' ') AS reviews
    FROM (
        SELECT DISTINCT title, author, publish_year, genre_consolidated, description
        FROM books_info
    ) b
    JOIN top_reviews t ON b.title = t.title
    GROUP BY
        b.title, b.author, b.publish_year, b.genre_consolidated, b.description
  """

books_info = pd.read_sql_query(query, conn)

<ipython-input-10-34a3fa8df5fa>:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  books_info = pd.read_sql_query(query, conn)


In [11]:
curr.close()
conn.close()

In [12]:
books_info.head()

,title,author,publish_year,genre_consolidated,description,reviews
0,1000,Christiane Stakenbrock,1999.0,Fiction / Literary,Overzicht tot en met de negentiende eeuw.,"Salem Kirban's only attempts at fiction, &quot..."
1,"10,000 Dreams and Their Traditional Meanings",Foulsham Staff,1995.0,"Body, Mind & Spirit / General","Alphabetical listing of brief, one-word dream ...",This is a great book. It shows dependable info...
2,"10,000 Dreams Interpreted or What's in a Dream",Gustavus Hindman Miller,1988.0,Psychology / General,"Examines the symbolism of 10,000 different dre...",I have used it a million times to interpret dr...
3,"10,000 White Horses",Betsy B. Lee,2002.0,Juvenile Fiction / Nursery Rhymes,Enjoy the white horses of the foam in the gall...,"What a fun, vivid story full of summer fun and..."
4,1000 Best Bartender Recipes,Suzi Parker,2005.0,Cooking / General,Go from novice mixer to expert in no time Lear...,I am a big fan of this bartender 1000 best rec...


### AWS BEDROCK: Load and Test LLM Model

In [13]:
# Pick a model
model_name_aws = "mistral.mistral-7b-instruct-v0:2"

In [14]:
# Initialize ExplainBot instance
explainbot = ExplainBot(model_name=model_name_aws)

In [15]:
# Generate the recommendation explanation
user_id = input_data["user_data"]["instances"][0]["user_id"]
recommended_book, cosine_similarity = input_data["recommendation"][0]

# Start time tracking
start_time = time.time()

try:
    # Get explanation for the recommended book using the updated explainbot instance
    explanation = explainbot.chat_recommendation_explanation(input_data, books_info)
except Exception as e:
    print(f"Error generating explanation: {e}")
    explanation = "Error: Unable to generate explanation."

# End time tracking
end_time = time.time()

# Calculate the time taken in seconds
time_taken = end_time - start_time
# Print the explanation for this user
print(f"Why '{recommended_book}' was recommended:\n")

if isinstance(explanation, dict):  # If the explanation is a dictionary
    for key, value in explanation.items():
        print(f"{key}: {value}\n")
else:
    print(explanation)

# Print the time taken
print(f"Time taken: {time_taken:.2f} seconds")

Why 'Tempting Fate' was recommended:

recommended_book: Tempting Fate

author: Jane Green

description: What is a woman's greatest temptation? How far will she go to find fulfillment—and how much is she willing to lose? Tempting Fate is an unforgettable, enthralling novel about the risks and rewards of "having it all" from beloved New York Times bestselling author Jane Green. Gabby and Elliott have been happily married for eighteen years. They have two daughters. They have a beautiful, loving home. Forty-three-year-old Gabby is the last person to have an affair. She can't relate to the way her friends desperately try to cling to the beauty and allure of their younger years. And yet she too knows her youth is quickly slipping away. She could never imagine how good it would feel to have a handsome younger man show interest in her...until the night it happens. Matt makes Gabby feel sparkling, fascinating, alive—something she hasn't felt in years. What begins as a long-distance friendship 